### 컴퓨터 비전 : OCR 개발 파이프라인

- 이 노트북은 **`OCR` 개발 파이프라인 예제 실습**을 수행하는 노트북입니다.

##### TrOCR를 활용한 OCR 개발 파이프라인 예제
   1.  사전학습된 TrOCR 모델 Fine-Tuning 및 저장 (Cell 5개)
   2.  Fine-tuning 한 모델의 성능 평가 (Cell 9개)

### 01. 사전학습된 TrOCR 모델 Fine-Tuning 및 저장

사전학습된 TrOCR 모델(`ddobokki/ko-trocr`)을 AIHub 한국어 OCR 데이터셋으로 Fine-tuning합니다.

AIHub 데이터 전처리부터 모델 학습까지 전체 파이프라인을 구현하여 한국어 손글씨 및 인쇄체 텍스트 인식 성능을 향상시킵니다.

* AIHub OCR 데이터셋 전처리 클래스로 인쇄체/필기체 JSON 라벨과 이미지를 매칭하여 CSV 형태로 변환
* 사전학습된 TrOCR 모델 로드, 한국어 토큰 추가 및 인코더 동결하여 디코더만 학습 설정  
* 이미지-텍스트 쌍 데이터를 Hugging Face Dataset 형식으로 변환하고 배치 단위 전처리 수행
* Seq2SeqTrainer를 사용하여 모델 Fine-tuning 실행 및 정확도 기반 성능 평가
* 학습된 모델과 프로세서를 지정 경로에 저장하고 TensorBoard 로깅으로 학습 과정 모니터링

In [1]:
# ============================================ 
# Cell 1: 라이브러리 임포트 
# ============================================

# 필요한 라이브러리 임포트
from transformers import TrOCRProcessor, VisionEncoderDecoderModel  # Hugging Face의 TrOCR 모델과 프로세서
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments  # 시퀀스-투-시퀀스 학습을 위한 트레이너
from datasets import Dataset, DatasetDict  # Hugging Face datasets 라이브러리
import torch  # PyTorch 딥러닝 프레임워크
from PIL import Image  # 이미지 처리를 위한 Pillow 라이브러리
import numpy as np  # 수치 연산을 위한 NumPy
import os  # 운영체제 관련 기능
import json  # JSON 파일 처리
import pandas as pd  # 데이터프레임 처리
from pathlib import Path  # 경로 처리를 위한 pathlib
import shutil  # 파일 복사 등 파일 시스템 작업
from tqdm import tqdm  # 진행 상황 표시를 위한 프로그레스 바

print("라이브러리 임포트 완료!")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

라이브러리 임포트 완료!
PyTorch 버전: 2.7.1+cu128
CUDA 사용 가능: True


In [3]:
# ============================================ 
# Cell 2: AIHub 데이터 전처리 클래스 정의 
# ============================================

class AIHubOCRPreprocessor:
    """
    AIHub OCR 데이터를 TrOCR 학습용으로 변환하는 전처리 클래스
    
    AIHub에서 제공하는 한국어 OCR 데이터셋은 다음과 같은 구조를 가집니다:
    - Training/Validation 폴더로 분리
    - 각 폴더 내에 인쇄체와 필기체 데이터 존재
    - 라벨 데이터(JSON)와 원천 데이터(이미지)가 별도 폴더에 저장
    
    이 클래스는 이러한 구조를 파싱하여 TrOCR 학습에 적합한 형태로 변환합니다.
    """
    
    def __init__(self, base_path):
        """
        전처리기 초기화
        
        Args:
            base_path (str): AIHub 데이터셋의 최상위 경로
        """
        self.base_path = Path(base_path)
        
    def extract_text_from_json(self, json_path):
        """
        JSON 라벨 파일에서 텍스트 정보를 추출하는 메서드
        
        AIHub JSON 파일은 다양한 형식을 가질 수 있습니다:
        1. 글자 타입: 'letter' 필드에 단일 문자 저장
        2. 단어 타입: 'word' 필드에 여러 글자의 배열 저장
        3. output 필드: 전체 텍스트가 저장된 경우
        
        Args:
            json_path (str or Path): JSON 파일 경로
            
        Returns:
            str or None: 추출된 텍스트 또는 None
        """
        try:
            # UTF-8 인코딩으로 JSON 파일 읽기
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # 'text' 필드가 없으면 None 반환
            if 'text' not in data:
                return None
                
            text_data = data['text']
            
            # 케이스 1: 글자 타입 처리 (letter 필드)
            if 'letter' in text_data:
                letter_data = text_data['letter']
                
                # letter가 딕셔너리인 경우
                if isinstance(letter_data, dict):
                    # value 키가 있으면 해당 값 반환
                    if 'value' in letter_data:
                        return str(letter_data['value'])
                    # value가 없으면 output 필드 확인
                    elif 'output' in text_data:
                        return str(text_data['output'])
                        
                # letter가 직접 문자열인 경우
                elif isinstance(letter_data, str):
                    return letter_data
            
            # 케이스 2: 단어 타입 처리 (word 필드)
            elif 'word' in text_data:
                words = []
                # word 배열의 각 요소에서 value 추출
                for word_info in text_data['word']:
                    if 'value' in word_info:
                        words.append(word_info['value'])
                # 모든 단어를 공백으로 연결하여 반환
                return ' '.join(words)
            
            # 케이스 3: output 필드가 직접 있는 경우 (폴백 옵션)
            elif 'output' in text_data:
                return str(text_data['output'])
            
            # 어떤 케이스에도 해당하지 않으면 None 반환
            return None
            
        except Exception as e:
            # 에러 발생 시 경로와 에러 메시지 출력
            print(f"Error processing {json_path}: {e}")
            return None
    
    def create_dataset(self, split='Training', output_dir='trocr_dataset'):
        """
        전체 데이터셋을 생성하는 메인 메서드
        
        이 메서드는 다음 작업을 수행합니다:
        1. 인쇄체와 필기체 데이터를 각각 처리
        2. JSON 라벨과 이미지 파일을 매칭
        3. 텍스트 추출 및 이미지 복사
        4. CSV 파일로 메타데이터 저장
        
        Args:
            split (str): 'Training' 또는 'Validation'
            output_dir (str): 출력 디렉토리 경로
            
        Returns:
            pandas.DataFrame: 생성된 데이터셋의 메타데이터
        """
        # 출력 디렉토리 생성
        output_path = Path(output_dir)
        output_path.mkdir(exist_ok=True)
        
        # 이미지 저장 폴더 생성 (images/training 또는 images/validation)
        images_dir = output_path / 'images' / split.lower()
        images_dir.mkdir(parents=True, exist_ok=True)
        
        # 데이터 수집을 위한 리스트 초기화
        data_list = []
        
        # ===== 인쇄체 데이터 처리 =====
        print(f"\n{split} 인쇄체 처리 중...")
        
        # 인쇄체 라벨과 원천 데이터 경로 설정
        printed_label_dir = self.base_path / split / f"[라벨]{split}_인쇄체" / "form"
        printed_source_dir = self.base_path / split / f"[원천]{split}_인쇄체" / "form"
        
        if printed_label_dir.exists():
            # 모든 하위 폴더 (001, 002, ...) 순회
            for folder in sorted(printed_label_dir.iterdir()):
                if folder.is_dir():
                    print(f"   폴더 {folder.name} 처리 중...")
                    # 현재 폴더의 모든 JSON 파일 찾기
                    json_files = list(folder.glob('*.json'))
                    
                    # 각 JSON 파일 처리 (진행 상황 표시)
                    for json_file in tqdm(json_files, desc=f"   {folder.name}"):
                        # 대응하는 원천 이미지 찾기
                        img_folder = printed_source_dir / folder.name
                        # 먼저 .jpg 확인
                        img_path = img_folder / (json_file.stem + '.jpg')
                        
                        # .jpg가 없으면 .png 확인
                        if not img_path.exists():
                            img_path = img_folder / (json_file.stem + '.png')
                        
                        # 이미지가 없으면 건너뛰기
                        if not img_path.exists():
                            continue
                        
                        # JSON에서 텍스트 추출
                        text = self.extract_text_from_json(json_file)
                        # 텍스트가 없거나 빈 문자열이면 건너뛰기
                        if not text or (isinstance(text, str) and text.strip() == ''):
                            continue
                        
                        # 이미지를 새 위치로 복사 (체계적인 이름으로)
                        new_img_name = f"printed_{folder.name}_{json_file.stem}.jpg"
                        new_img_path = images_dir / new_img_name
                        shutil.copy2(img_path, new_img_path)
                        
                        # 메타데이터 추가
                        data_list.append({
                            'image_path': f'images/{split.lower()}/{new_img_name}',
                            'text': text,
                            'type': 'printed',  # 인쇄체 표시
                            'folder': folder.name,
                            'original_file': json_file.stem
                        })
        
        # ===== 필기체 데이터 처리 =====
        print(f"\n{split} 필기체 처리 중...")
        
        # 필기체는 '1.글자'와 '2.단어' 두 가지 서브타입으로 구분
        for sub_type in ['1.글자', '2.단어']:
            # 필기체 라벨과 원천 데이터 경로 설정
            handwritten_label_dir = self.base_path / split / f"[라벨]{split}_필기체" / sub_type
            handwritten_source_dir = self.base_path / split / f"[원천]{split}_필기체" / sub_type
            
            # 디렉토리 존재 확인
            if not handwritten_label_dir.exists():
                print(f"   경고: {sub_type} 라벨 폴더가 없습니다: {handwritten_label_dir}")
                continue
                
            if not handwritten_source_dir.exists():
                print(f"   경고: {sub_type} 원천 폴더가 없습니다: {handwritten_source_dir}")
                continue
                
            print(f"   {sub_type} 처리 중...")
            
            # 처리 통계를 위한 카운터
            sub_type_count = 0  # 성공적으로 처리된 파일 수
            failed_count = 0    # 실패한 파일 수
            
            # 각 하위 폴더 처리
            for folder in sorted(handwritten_label_dir.iterdir()):
                if folder.is_dir():
                    json_files = list(folder.glob('*.json'))
                    print(f"      폴더 {folder.name} 처리 중... ({len(json_files)}개 파일)")
                    
                    # JSON 파일이 없는 경우 경고
                    if len(json_files) == 0:
                        print(f"         JSON 파일이 없습니다!")
                        continue
                    
                    # 폴더별 처리 통계
                    folder_success = 0
                    folder_failed = 0
                    
                    # 각 JSON 파일 처리
                    for json_file in tqdm(json_files, desc=f"      {folder.name}"):
                        # 대응하는 원천 이미지 폴더 확인
                        img_folder = handwritten_source_dir / folder.name
                        
                        if not img_folder.exists():
                            folder_failed += 1
                            failed_count += 1
                            continue
                        
                        # 이미지 파일 찾기
                        img_path = img_folder / (json_file.stem + '.jpg')
                        if not img_path.exists():
                            img_path = img_folder / (json_file.stem + '.png')
                        
                        if not img_path.exists():
                            folder_failed += 1
                            failed_count += 1
                            continue
                        
                        # 텍스트 추출
                        text = self.extract_text_from_json(json_file)
                        if not text or (isinstance(text, str) and text.strip() == ''):
                            folder_failed += 1
                            failed_count += 1
                            continue
                        
                        # 이미지 복사 및 메타데이터 저장
                        try:
                            # 파일명에서 점(.)을 언더스코어로 변경 (파일시스템 호환성)
                            sub_type_clean = sub_type.replace('.', '_')
                            new_img_name = f"handwritten_{sub_type_clean}_{folder.name}_{json_file.stem}.jpg"
                            new_img_path = images_dir / new_img_name
                            shutil.copy2(img_path, new_img_path)
                            
                            # 메타데이터 추가
                            data_list.append({
                                'image_path': f'images/{split.lower()}/{new_img_name}',
                                'text': text,
                                'type': 'handwritten',  # 필기체 표시
                                'sub_type': sub_type,   # 글자/단어 구분
                                'folder': folder.name,
                                'original_file': json_file.stem
                            })
                            
                            sub_type_count += 1
                            folder_success += 1
                        except Exception as e:
                            print(f"         처리 실패 {json_file.name}: {e}")
                            folder_failed += 1
                            failed_count += 1
                    
                    # 폴더별 처리 결과 출력
                    print(f"         완료: 성공 {folder_success}개, 실패 {folder_failed}개")
            
            # 서브타입별 전체 처리 결과 출력
            print(f"      {sub_type} 전체 처리 완료: {sub_type_count}개 파일 (실패: {failed_count}개)")
        
        # ===== CSV 저장 및 통계 출력 =====
        # 데이터프레임 생성
        df = pd.DataFrame(data_list)
        
        # CSV 파일로 저장 (UTF-8 인코딩)
        csv_path = output_path / f'{split.lower()}.csv'
        df.to_csv(csv_path, index=False, encoding='utf-8')
        
        # 처리 결과 통계 출력
        print(f"\n{split} 데이터셋 생성 완료!")
        print(f"   - 총 샘플 수: {len(df)}")
        
        # 데이터가 있는 경우에만 상세 통계 출력
        if len(df) > 0:
            print(f"   - 인쇄체: {len(df[df['type'] == 'printed'])}")
            print(f"   - 필기체: {len(df[df['type'] == 'handwritten'])}")
            
            # 필기체 세부 통계
            if 'handwritten' in df['type'].values:
                handwritten_df = df[df['type'] == 'handwritten']
                if 'sub_type' in handwritten_df.columns:
                    print(f"     - 글자: {len(handwritten_df[handwritten_df['sub_type'] == '1.글자'])}")
                    print(f"     - 단어: {len(handwritten_df[handwritten_df['sub_type'] == '2.단어'])}")
        else:
            print("   경고: 데이터가 없습니다! 경로를 확인해주세요.")
            print(f"   확인할 경로: {self.base_path}")
        
        print(f"   - CSV 위치: {csv_path}")
        
        return df

print("AIHubOCRPreprocessor 클래스 정의 완료!")

AIHubOCRPreprocessor 클래스 정의 완료!


In [ ]:
# ============================================ 
# Cell 3: 데이터 전처리 실행 (선택사항)
# ============================================

# 주의: 이 셀은 AIHub 데이터가 존재하고 전처리를 진행하지 않은 경우에만 실행하세요.
# 이미 전처리 데이터로 trorcr_dataset.zip의 압축을 풀어서 활용하셔도 됩니다.
# 이미 전처리된 데이터가 있다면 이 셀을 건너뛰고 Cell 5로 진행하세요.

# AIHub 데이터 경로 설정 (본인의 경로로 수정)
AIHUB_DATA_PATH = r'C:\Users\SSAFY\Downloads\다양한 형태의 한글 문자 OCR'  # 실제 경로로 변경하세요

# 전처리기 초기화
print("데이터 전처리 시작...")
preprocessor = AIHubOCRPreprocessor(AIHUB_DATA_PATH)

# Training 데이터셋 전처리
print("\n=== Training 데이터셋 전처리 ===")
train_df = preprocessor.create_dataset(split='Training', output_dir='trocr_dataset')

# Validation 데이터셋 전처리
print("\n=== Validation 데이터셋 전처리 ===")
val_df = preprocessor.create_dataset(split='Validation', output_dir='trocr_dataset')

print("\n전처리 완료!")
print("생성된 파일:")
print("- trocr_dataset/training.csv")
print("- trocr_dataset/validation.csv")
print("- trocr_dataset/images/training/")
print("- trocr_dataset/images/validation/")

데이터 전처리 시작...

=== Training 데이터셋 전처리 ===

Training 인쇄체 처리 중...
   폴더 001 처리 중...


   001: 100%|██████████| 1200/1200 [00:01<00:00, 802.80it/s]


   폴더 002 처리 중...


   002: 100%|██████████| 1200/1200 [00:01<00:00, 692.77it/s]


   폴더 003 처리 중...


...

         완료: 성공 4473개, 실패 0개
      폴더 141 처리 중... (4660개 파일)


      141: 100%|██████████| 4660/4660 [00:05<00:00, 929.54it/s]


         완료: 성공 4660개, 실패 0개
      폴더 142 처리 중... (5195개 파일)


      142: 100%|██████████| 5195/5195 [00:03<00:00, 1350.24it/s]


         완료: 성공 5195개, 실패 0개
      2.단어 전체 처리 완료: 24281개 파일 (실패: 0개)

Validation 데이터셋 생성 완료!
   - 총 샘플 수: 35493
   - 인쇄체: 750
   - 필기체: 34743
     - 글자: 10462
     - 단어: 24281
   - CSV 위치: trocr_dataset\validation.csv

전처리 완료!
생성된 파일:
- trocr_dataset/training.csv
- trocr_dataset/validation.csv
- trocr_dataset/images/training/
- trocr_dataset/images/validation/


In [23]:
# ============================================ 
# Cell 4: KoreanTrOCRTrainer 클래스 정의 
# ============================================

class KoreanTrOCRTrainer:
    """
    AIHub 데이터로 한국어 TrOCR을 학습하는 트레이너 클래스
    
    TrOCR은 이미지에서 텍스트를 추출하는 Transformer 기반 모델입니다.
    이 클래스는 영어로 사전학습된 TrOCR 모델을 한국어로 Fine-tuning합니다.
    
    주요 기능:
    1. 한국어 토큰 추가
    2. 데이터셋 준비 및 전처리
    3. 모델 학습
    """
    
    def __init__(self, model_name="ddobokki/ko-trocr", freeze_encoder=True):
        """
        트레이너 초기화
        
        Args:
            model_name (str): 사용할 사전학습 모델 이름
                - "ddobokki/ko-trocr": 한국어 손글씨 특화 모델 (base, 권장)
                - "team-lucid/trocr-small-korean": 한국어 인쇄체 특화 모델 (small)
                - "microsoft/trocr-base-stage1": 영어 기본 모델
                - "microsoft/trocr-large-stage1": 영어 대용량 모델
            freeze_encoder (bool): 인코더 동결 여부 (기본값: True)
                - True: 인코더 가중치 고정, 디코더만 학습 (권장)
                - False: 전체 모델 학습
        """
        print("모델 초기화 중...")
        
        # GPU 사용 가능 여부 확인 및 디바이스 설정
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"   디바이스: {self.device}")
        
        # TrOCR 프로세서 로드 (이미지 전처리 + 텍스트 토크나이저)
        self.processor = TrOCRProcessor.from_pretrained(model_name)
        
        # TrOCR 모델 로드 (Vision Encoder + Text Decoder)
        self.model = VisionEncoderDecoderModel.from_pretrained(model_name)

        if freeze_encoder:
            print("인코더 레이어를 동결합니다...")
            for param in self.model.encoder.parameters():
                param.requires_grad = False
            
            # 동결된 파라미터 수 확인
            trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
            total_params = sum(p.numel() for p in self.model.parameters())
            print(f"학습 가능한 파라미터: {trainable_params:,} / {total_params:,} ({trainable_params/total_params*100:.1f}%)")
        
        # 모델을 지정된 디바이스로 이동
        self.model.to(self.device)
        
    def prepare_dataset(self, train_csv, val_csv, max_samples=None):
        """
        학습용 데이터셋을 준비하는 메서드
        
        CSV 파일을 읽어서 Hugging Face Dataset 형식으로 변환하고,
        이미지와 텍스트를 모델 입력 형식에 맞게 전처리합니다.
        
        Args:
            train_csv (str): 학습 데이터 CSV 경로
            val_csv (str): 검증 데이터 CSV 경로
            max_samples (int, optional): 사용할 최대 샘플 수 (디버깅용)
            
        Returns:
            tuple: (train_dataset, val_dataset)
        """
        print("\n데이터셋 준비 중...")
        
        # CSV 파일 읽기
        train_df = pd.read_csv(train_csv)
        val_df = pd.read_csv(val_csv)
        
        # 샘플 수 제한 (선택사항, 빠른 테스트용)
        if max_samples:
            train_df = train_df.sample(n=min(max_samples, len(train_df)), random_state=42)
            val_df = val_df.sample(n=min(max_samples//5, len(val_df)), random_state=42)
        
        print(f"   학습 샘플: {len(train_df)}")
        print(f"   검증 샘플: {len(val_df)}")
        
        # Pandas DataFrame을 Hugging Face Dataset으로 변환
        train_dataset = Dataset.from_pandas(train_df)
        val_dataset = Dataset.from_pandas(val_df)
        
        def preprocess_function(examples):
            """
            배치 단위로 데이터를 전처리하는 내부 함수
            
            이미지를 로드하고 크기를 조정한 후,
            모델 입력 형식(pixel_values, labels)으로 변환합니다.
            
            Args:
                examples: 배치 데이터 (딕셔너리 형태)
                
            Returns:
                dict: 전처리된 데이터
            """
            images = []
            # CSV가 있는 디렉토리를 기준으로 상대 경로 해석
            base_dir = os.path.dirname(train_csv)
            
            # 배치의 각 이미지 로드
            for img_path in examples['image_path']:
                try:
                    # 전체 경로 생성
                    full_path = os.path.join(base_dir, img_path)
                    # RGB 모드로 이미지 열기
                    image = Image.open(full_path).convert("RGB")
                    
                    # 이미지 크기 조정 (메모리 절약 및 학습 효율성)
                    max_size = 384  # TrOCR 기본 입력 크기
                    if max(image.size) > max_size:
                        # 비율 유지하며 크기 조정
                        ratio = max_size / max(image.size)
                        new_size = tuple(int(dim * ratio) for dim in image.size)
                        image = image.resize(new_size, Image.Resampling.LANCZOS)
                    
                    images.append(image)
                except Exception as e:
                    print(f"이미지 로드 실패: {img_path}, {e}")
                    # 로드 실패 시 빈 이미지로 대체 (학습 안정성)
                    images.append(Image.new("RGB", (384, 384), "white"))
            
            # 이미지를 모델 입력 형식으로 변환 (정규화 포함)
            pixel_values = self.processor(images, return_tensors="pt").pixel_values
            
            # 텍스트를 토큰화하여 labels 생성
            labels = self.processor.tokenizer(
                examples['text'],
                padding="max_length",     # 최대 길이까지 패딩
                max_length=64,           # 한국어는 보통 짧으므로 64로 설정
                truncation=True,         # 긴 텍스트는 자르기
                return_tensors="pt"
            ).input_ids
            
            # 중요: 패딩 토큰을 -100으로 변경
            # -100은 PyTorch에서 loss 계산 시 무시되는 특수 값
            labels[labels == self.processor.tokenizer.pad_token_id] = -100
            
            # 전처리된 데이터 반환
            encoding = {
                "pixel_values": pixel_values,
                "labels": labels
            }
            
            return encoding
        
        # 전처리 함수를 데이터셋에 적용
        print("   전처리 중...")
        
        # 학습 데이터 전처리
        train_dataset = train_dataset.map(
            preprocess_function,
            batched=True,                        # 배치 단위로 처리
            batch_size=4,                        # 배치 크기 (메모리에 따라 조정)
            remove_columns=train_dataset.column_names,  # 원본 컬럼 제거
            writer_batch_size=100,               # 디스크 쓰기 배치 크기
            keep_in_memory=False,                # 메모리 대신 디스크 사용
            load_from_cache_file=True,           # 캐시 활용
            desc="Training dataset 전처리"
        )
        
        # 검증 데이터 전처리
        val_dataset = val_dataset.map(
            preprocess_function,
            batched=True,
            batch_size=8,
            remove_columns=val_dataset.column_names,
            writer_batch_size=100,
            keep_in_memory=False,
            load_from_cache_file=True,
            desc="Validation dataset 전처리"
        )
        
        # PyTorch 텐서 형식으로 설정
        train_dataset.set_format(type="torch", columns=["pixel_values", "labels"])
        val_dataset.set_format(type="torch", columns=["pixel_values", "labels"])
        
        return train_dataset, val_dataset
    
    def compute_metrics(self, eval_preds):
        """
        평가 메트릭을 계산하는 메서드
        
        모델의 예측값과 정답을 비교하여 정확도를 계산합니다.
        
        Args:
            eval_preds: (predictions, labels) 튜플
            
        Returns:
            dict: 계산된 메트릭
        """
        preds, labels = eval_preds
        
        # 예측값이 튜플인 경우 첫 번째 요소 사용
        if isinstance(preds, tuple):
            preds = preds[0]
        
        # -100인 토큰(패딩)은 pad_token_id로 변경
        preds = np.where(preds != -100, preds, self.processor.tokenizer.pad_token_id)
        
        # 토큰 ID를 텍스트로 디코딩
        decoded_preds = self.processor.tokenizer.batch_decode(preds, skip_special_tokens=True)
        
        # 레이블도 동일하게 처리
        labels = np.where(labels != -100, labels, self.processor.tokenizer.pad_token_id)
        decoded_labels = self.processor.tokenizer.batch_decode(labels, skip_special_tokens=True)
        
        # 정확도 계산: 예측과 정답이 완전히 일치하는 샘플의 비율
        exact_match = sum([pred.strip() == label.strip() 
                          for pred, label in zip(decoded_preds, decoded_labels)])
        accuracy = exact_match / len(decoded_labels) if len(decoded_labels) > 0 else 0
        
        return {"accuracy": accuracy}
    
    def train(self, train_dataset, val_dataset, output_dir="korean-trocr", 
              epochs=10, batch_size=8, learning_rate=5e-5):
        """
        모델 학습을 수행하는 메서드
        
        Seq2SeqTrainer를 사용하여 모델을 학습하고,
        주기적으로 검증 및 체크포인트 저장을 수행합니다.
        
        Args:
            train_dataset: 학습 데이터셋
            val_dataset: 검증 데이터셋
            output_dir (str): 모델 저장 경로
            epochs (int): 학습 에폭 수
            batch_size (int): 배치 크기
            learning_rate (float): 학습률
            
        Returns:
            Seq2SeqTrainer: 학습된 트레이너 객체
        """
        print(f"\n학습 시작!")
        print(f"   에폭: {epochs}")
        print(f"   배치 크기: {batch_size}")
        print(f"   학습률: {learning_rate}")
        
        def custom_data_collator(features):
            """
            TrOCR용 커스텀 데이터 콜레이터
            
            배치의 각 샘플을 하나의 텐서로 결합합니다.
            
            Args:
                features: 배치 샘플 리스트
                
            Returns:
                dict: 결합된 배치 데이터
            """
            batch = {}
            
            # pixel_values 스태킹
            if 'pixel_values' in features[0]:
                pixel_values = torch.stack([f['pixel_values'] for f in features])
                batch['pixel_values'] = pixel_values
            
            # labels 스태킹
            if 'labels' in features[0]:
                labels = torch.stack([f['labels'] for f in features])
                batch['labels'] = labels
                
            return batch
        
        # 학습 설정 정의
        training_args = Seq2SeqTrainingArguments(
            output_dir=output_dir,               # 출력 디렉토리
            num_train_epochs=epochs,             # 전체 에폭 수
            per_device_train_batch_size=batch_size,  # GPU당 학습 배치 크기
            per_device_eval_batch_size=batch_size,   # GPU당 평가 배치 크기
            warmup_steps=1000,                   # 학습률 웜업 스텝
            learning_rate=learning_rate,         # 최대 학습률
            logging_steps=100,                   # 로깅 주기
            save_steps=1000,                     # 체크포인트 저장 주기
            eval_steps=1000,                     # 평가 주기
            eval_strategy="steps",               # 평가 전략 (steps 또는 epoch)
            save_total_limit=3,                  # 최대 체크포인트 개수
            predict_with_generate=True,          # 생성 모드로 예측
            fp16=torch.cuda.is_available(),      # GPU 사용 시 16비트 연산
            push_to_hub=False,                   # Hugging Face Hub 업로드 여부
            report_to=["tensorboard"],           # 로깅 도구
            load_best_model_at_end=True,         # 학습 종료 시 최고 모델 로드
            metric_for_best_model="eval_loss",   # 최고 모델 선택 기준
            greater_is_better=False,             # 낮을수록 좋음 (loss)
            generation_max_length=64,            # 생성 최대 길이
            generation_num_beams=4,              # 빔 서치 빔 개수
        )
        
        # Seq2SeqTrainer 생성
        trainer = Seq2SeqTrainer(
            model=self.model,                    # 학습할 모델
            args=training_args,                  # 학습 설정
            train_dataset=train_dataset,         # 학습 데이터셋
            eval_dataset=val_dataset,            # 검증 데이터셋
            processing_class=self.processor,     # 전처리기
            data_collator=custom_data_collator,  # 데이터 콜레이터
            compute_metrics=self.compute_metrics,  # 메트릭 계산 함수
        )
        
        # 학습 실행
        trainer.train()
        
        # 모델과 프로세서 저장
        print(f"\n모델 저장 중...")
        trainer.save_model()
        self.processor.save_pretrained(output_dir)
        
        print(f"학습 완료! 모델 저장 위치: {output_dir}")
        
        return trainer

print("KoreanTrOCRTrainer 클래스 정의 완료!")

KoreanTrOCRTrainer 클래스 정의 완료!


In [24]:
# ============================================ 
# Cell 5: 모델 학습 실행
# ============================================

print("=" * 50)
print("TrOCR 학습 시작")
print("=" * 50)

# 트레이너 초기화
# 사용 가능한 모델:
# - "ddobokki/ko-trocr": 한국어 손글씨 특화 (권장)
# - "team-lucid/trocr-small-korean": 한국어 인쇄체 특화
# - "microsoft/trocr-base-stage1": 영어 기본 모델
trainer = KoreanTrOCRTrainer(
    model_name="ddobokki/ko-trocr",  # 사용할 모델
    freeze_encoder=True              # 인코더 동결 (권장)
)

# 데이터셋 준비
print("\n데이터셋 로드 중...")
train_dataset, val_dataset = trainer.prepare_dataset(
    train_csv='trocr_dataset/training.csv',    # 학습 데이터 CSV
    val_csv='trocr_dataset/validation.csv',    # 검증 데이터 CSV
    max_samples=500  # 빠른 테스트용 (실제 학습 시에는 None으로 설정)
)

# 모델 학습
print("\n학습 시작...")
trained_model = trainer.train(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    output_dir='aihub-korean-trocr',     # 모델 저장 경로
    epochs=1,                            # 에폭 수 (실제로는 30+ 권장)
    batch_size=2 if torch.cuda.is_available() else 1,  # 배치 크기
    learning_rate=1e-6                   # 학습률
)

print("=" * 50)
print("학습 완료!")
print("=" * 50)

TrOCR 학습 시작
모델 초기화 중...
   디바이스: cuda
인코더 레이어를 동결합니다...
학습 가능한 파라미터: 127,039,784 / 213,693,224 (59.4%)

데이터셋 로드 중...

데이터셋 준비 중...
   학습 샘플: 500
   검증 샘플: 100
   전처리 중...


Training dataset 전처리:   0%|          | 0/500 [00:00<?, ? examples/s]

Validation dataset 전처리:   0%|          | 0/100 [00:00<?, ? examples/s]


학습 시작...

학습 시작!
   에폭: 1
   배치 크기: 2
   학습률: 1e-06


Step,Training Loss,Validation Loss



모델 저장 중...
학습 완료! 모델 저장 위치: aihub-korean-trocr
학습 완료!


### 02. Fine-tuning 한 모델의 성능 평가

학습된 TrOCR 모델의 성능을 다각도로 평가하고 오류 패턴을 분석합니다.

반복 패턴 방지를 위한 개선된 생성 설정과 상세한 시각화를 통해 모델 성능을 종합적으로 진단합니다.

* 학습된 TrOCR 모델 로드 및 이미지 전처리를 통한 품질 개선된 텍스트 예측 수행
* Character Error Rate(CER)와 Word Error Rate(WER) 메트릭 계산으로 정확한 성능 측정
* 인쇄체/필기체 타입별 성능 분석 및 텍스트 길이별 오류 분포 통계 생성
* 반복 패턴 감지 및 최악 오류 케이스 분석을 통한 상세한 오류 패턴 진단

In [34]:
# ============================================ 
# Cell 1: 라이브러리 임포트 및 환경 설정
# ============================================

# 필수 라이브러리 임포트
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image, ImageEnhance
import torch
import pandas as pd
import os
from tqdm import tqdm
import Levenshtein
from collections import defaultdict
import evaluate  # Hugging Face evaluate 라이브러리
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 임포트 완료!")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

라이브러리 임포트 완료!
PyTorch 버전: 2.7.1+cu128
CUDA 사용 가능: True


In [35]:
# ============================================ 
# Cell 2: TrOCREvaluator 클래스 초기화 및 기본 메서드
# ============================================

class TrOCREvaluator:
    """
    TrOCR 모델 평가를 위한 클래스
    
    주요 기능:
    - 학습된 모델 로드 및 예측
    - Character Error Rate (CER) 및 Word Error Rate (WER) 계산
    - 타입별 성능 분석 (인쇄체/필기체)
    - 오류 패턴 분석
    """
    
    def __init__(self, model_path):
        """
        평가기 초기화
        
        Args:
            model_path (str): 학습된 모델이 저장된 디렉토리 경로
        """
        print(f"[모델 로딩] {model_path}")
        
        # 프로세서와 모델 로드
        self.processor = TrOCRProcessor.from_pretrained(model_path)
        self.model = VisionEncoderDecoderModel.from_pretrained(model_path)
        
        # GPU 사용 가능 여부 확인 및 디바이스 설정
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        self.model.eval()  # 평가 모드로 전환 (드롭아웃 비활성화 등)
        
        print(f"[모델 로드 완료] 디바이스: {self.device}")
        
        # CER 메트릭 로드 시도
        try:
            self.cer_metric = evaluate.load("cer")
            print("[CER 메트릭 로드 완료]")
        except:
            print("[경고] CER 메트릭 로드 실패. 수동 계산 모드로 전환")
            self.cer_metric = None
    
    def preprocess_image(self, image):
        """
        이미지 전처리 (품질 개선)
        
        손글씨 인식 성능 향상을 위한 전처리 단계
        
        Args:
            image (PIL.Image): 원본 이미지
            
        Returns:
            PIL.Image: 전처리된 이미지
        """
        # 대비 향상 (손글씨가 흐릿한 경우 도움)
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.3)
        
        # 선명도 향상
        enhancer = ImageEnhance.Sharpness(image)
        image = enhancer.enhance(1.2)
        
        return image
    
    def normalize_text(self, text, remove_spaces=True):
        """
        텍스트 정규화 함수
        
        Args:
            text (str): 원본 텍스트
            remove_spaces (bool): 공백 제거 여부
            
        Returns:
            str: 정규화된 텍스트
        """
        import unicodedata
        
        # 기본 정규화
        text = str(text).strip()
        
        # 유니코드 정규화 (NFC)
        text = unicodedata.normalize('NFC', text)
        
        if remove_spaces:
            # 한글 음절 사이의 공백 제거
            # 예: "초 저 녁" -> "초저녁"
            import re
            # 한글 문자 사이의 모든 공백 제거
            text = re.sub(r'(?<=[가-힣])\s+(?=[가-힣])', '', text)
            # 여러 공백을 하나로
            text = re.sub(r'\s+', ' ', text)
            
        # 보이지 않는 문자 제거
        text = ''.join(char for char in text if char.isprintable() or char in '\n\r\t')
        
        return text

print("TrOCREvaluator 클래스 기본 메서드 정의 완료!")

TrOCREvaluator 클래스 기본 메서드 정의 완료!


In [36]:
# ============================================ 
# Cell 3: 예측 및 메트릭 계산 메서드 추가
# ============================================

def predict(self, image_path, apply_preprocessing=True):
    """
    이미지에서 텍스트 예측
    
    반복 패턴 방지를 위한 개선된 생성 설정 포함
    
    Args:
        image_path (str): 이미지 파일 경로
        apply_preprocessing (bool): 이미지 전처리 적용 여부
        
    Returns:
        str: 예측된 텍스트
    """
    try:
        # 이미지 로드 및 RGB 변환
        image = Image.open(image_path).convert("RGB")
        
        # 전처리 적용 (선택적)
        if apply_preprocessing:
            image = self.preprocess_image(image)
        
        # 모델 입력을 위한 전처리
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        pixel_values = pixel_values.to(self.device)
        
        # 예측 수행 (그래디언트 계산 비활성화로 메모리 절약)
        with torch.no_grad():
            # 개선된 생성 설정
            generated_ids = self.model.generate(
                pixel_values,
                max_length=64,              # 최대 생성 길이
                num_beams=5,                # 빔 서치 크기 (품질 향상)
                no_repeat_ngram_size=2,     # 2-gram 반복 방지 (중요!)
                length_penalty=1.0,         # 길이 패널티 (1.0 = 중립)
                early_stopping=True,        # EOS 토큰 발견 시 조기 종료
                temperature=1.0,            # 샘플링 온도 (1.0 = 기본)
                do_sample=False,            # 결정적 디코딩 사용
                repetition_penalty=1.2      # 반복 패널티 (추가 안전장치)
            )
        
        # 생성된 토큰을 텍스트로 디코딩
        generated_text = self.processor.batch_decode(
            generated_ids, 
            skip_special_tokens=True  # [PAD], [EOS] 등 특수 토큰 제거
        )[0]
        
        # 후처리: 불필요한 공백 제거
        generated_text = generated_text.strip()
        
        return generated_text
        
    except Exception as e:
        print(f"[예측 오류] {image_path}: {e}")
        return ""

def calculate_cer(self, pred_text, true_text):
    """
    Character Error Rate (CER) 계산
    
    CER = (삽입 + 삭제 + 치환) / 전체 문자 수
    
    Args:
        pred_text (str): 예측된 텍스트
        true_text (str): 정답 텍스트
        
    Returns:
        float: CER 값 (0.0 ~ 1.0+)
    """
    # 빈 텍스트 처리
    if len(true_text) == 0:
        return 0.0 if len(pred_text) == 0 else 1.0
    
    # Levenshtein 거리 계산 (편집 거리)
    distance = Levenshtein.distance(pred_text, true_text)
    
    # CER = 편집 거리 / 정답 텍스트 길이
    return distance / len(true_text)

def calculate_wer(self, pred_text, true_text):
    """
    Word Error Rate (WER) 계산
    
    단어 단위로 오류율을 계산 (한국어의 경우 공백으로 분리)
    
    Args:
        pred_text (str): 예측된 텍스트
        true_text (str): 정답 텍스트
        
    Returns:
        float: WER 값 (0.0 ~ 1.0+)
    """
    # 공백 기준으로 단어 분리
    pred_words = pred_text.split()
    true_words = true_text.split()
    
    # 빈 텍스트 처리
    if len(true_words) == 0:
        return 0.0 if len(pred_words) == 0 else 1.0
    
    # 단어 시퀀스를 다시 문자열로 변환하여 거리 계산
    # (더 정교한 WER은 단어 단위 편집 거리를 계산해야 함)
    distance = Levenshtein.distance(' '.join(pred_words), ' '.join(true_words))
    return distance / len(' '.join(true_words))

# 클래스에 메서드 추가
TrOCREvaluator.predict = predict
TrOCREvaluator.calculate_cer = calculate_cer
TrOCREvaluator.calculate_wer = calculate_wer

print("예측 및 메트릭 계산 메서드 추가 완료!")

예측 및 메트릭 계산 메서드 추가 완료!


In [37]:
# ============================================ 
# Cell 4: 샘플 테스트 메서드 추가
# ============================================

def test_samples(self, test_csv, num_samples=10, return_df=True, normalize_spaces=True):
    """
    빠른 샘플 테스트 (DataFrame 반환 기능 추가)
    
    전체 평가 전 모델이 제대로 작동하는지 확인
    
    Args:
        test_csv (str): 테스트 데이터 CSV 파일 경로
        num_samples (int): 테스트할 샘플 수
        return_df (bool): 결과를 DataFrame으로 반환할지 여부
        normalize_spaces (bool): 공백 정규화 적용 여부
    
    Returns:
        dict: 평가 결과 딕셔너리 (return_df=True인 경우 detailed_results 포함)
    """
    print(f"\n[샘플 테스트] {num_samples}개 샘플 테스트 시작")
    
    # CSV 파일 로드
    df = pd.read_csv(test_csv)
    
    # 랜덤 샘플 선택 (재현 가능하도록 random_state 고정)
    samples = df.sample(n=min(num_samples, len(df)), random_state=42)
    
    # 테이블 헤더 출력
    print(f"\n{'='*100}")
    print(f"{'파일명':^30} | {'실제 텍스트':^30} | {'예측 텍스트':^30}")
    print(f"{'='*100}")
    
    # 기본 디렉토리 경로
    base_dir = os.path.dirname(test_csv)
    
    # 결과 저장을 위한 리스트 (DataFrame 반환용)
    results_list = []
    correct = 0  # 완전히 일치하는 예측 수
    
    # 각 샘플에 대해 예측 수행
    for _, row in samples.iterrows():
        # 이미지 경로 구성
        img_path = os.path.join(base_dir, row['image_path'])
        true_text = str(row['text'])
        
        # 예측 수행
        pred_text = self.predict(img_path)
        
        # 정규화 적용 (선택적)
        if normalize_spaces:
            true_text_normalized = self.normalize_text(true_text)
            pred_text_normalized = self.normalize_text(pred_text)
            is_correct = (pred_text_normalized == true_text_normalized)
        else:
            is_correct = (pred_text == true_text)
        
        # 메트릭 계산 (정규화된 텍스트로)
        cer = self.calculate_cer(pred_text_normalized if normalize_spaces else pred_text, 
                               true_text_normalized if normalize_spaces else true_text)
        wer = self.calculate_wer(pred_text_normalized if normalize_spaces else pred_text, 
                               true_text_normalized if normalize_spaces else true_text)
        
        # 긴 텍스트는 표시용으로 자르기
        true_display = true_text[:25] + '...' if len(true_text) > 25 else true_text
        pred_display = pred_text[:25] + '...' if len(pred_text) > 25 else pred_text
        
        # 정확도 체크
        if is_correct:
            correct += 1
            status = "[O]"  # 정답
        else:
            status = "[X]"  # 오답
            
            # 디버깅: 틀린 경우 상세 분석
            if true_display == pred_display:  # 보기에는 같은데 틀린 경우
                print(f"\n[디버깅] 숨겨진 차이 분석:")
                print(f"  실제 텍스트 길이: {len(true_text)}, 예측 텍스트 길이: {len(pred_text)}")
                print(f"  실제 텍스트 바이트: {true_text.encode('utf-8')}")
                print(f"  예측 텍스트 바이트: {pred_text.encode('utf-8')}")
                
                # 문자 단위 비교
                for i, (t, p) in enumerate(zip(true_text, pred_text)):
                    if t != p:
                        print(f"  위치 {i}: 실제='{t}'(U+{ord(t):04X}), 예측='{p}'(U+{ord(p):04X})")
                
                # 앞뒤 공백 확인
                if true_text != true_text.strip() or pred_text != pred_text.strip():
                    print(f"  공백 문제: 실제='{repr(true_text)}', 예측='{repr(pred_text)}'")
                print()
        
        # 결과 출력
        filename = os.path.basename(img_path)[:25]
        print(f"{status} {filename:^28} | {true_display:^30} | {pred_display:^30}")
        
        # DataFrame용 결과 저장
        if return_df:
            results_list.append({
                'file': row.get('original_file', os.path.basename(img_path)),
                'type': row['type'],
                'true_text': true_text,
                'pred_text': pred_text,
                'is_correct': is_correct,
                'cer': cer,
                'wer': wer,
                'text_length': len(true_text)
            })
    
    # 샘플 정확도 출력
    accuracy = correct / num_samples * 100
    print(f"\n[샘플 정확도] {correct}/{num_samples} ({accuracy:.1f}%)")
    
    # 평균 메트릭 계산
    if results_list:
        avg_cer = np.mean([r['cer'] for r in results_list])
        avg_wer = np.mean([r['wer'] for r in results_list])
        print(f"[평균 CER] {avg_cer:.4f}")
        print(f"[평균 WER] {avg_wer:.4f}")
    
    # 결과 반환
    result_dict = {
        'sample_accuracy': accuracy / 100,
        'sample_correct': correct,
        'sample_total': num_samples
    }
    
    if return_df and results_list:
        results_df = pd.DataFrame(results_list)
        result_dict['detailed_results'] = results_df
        result_dict['overall_cer'] = avg_cer
        result_dict['overall_wer'] = avg_wer
    else:
        result_dict['detailed_results'] = None
    
    return result_dict

# 클래스에 메서드 추가
TrOCREvaluator.test_samples = test_samples

print("샘플 테스트 메서드 추가 완료!")

샘플 테스트 메서드 추가 완료!


In [38]:
# ============================================ 
# Cell 5: 전체 평가 및 오류 분석 메서드 추가
# ============================================

def evaluate_full(self, test_csv, save_results=True, normalize_spaces=True):
    """
    전체 데이터셋 종합 평가
    
    모든 테스트 데이터에 대해 상세한 평가 수행
    
    Args:
        test_csv (str): 테스트 데이터 CSV 파일 경로
        save_results (bool): 상세 결과를 CSV로 저장할지 여부
        normalize_spaces (bool): 공백 정규화 적용 여부
        
    Returns:
        dict: 평가 결과 딕셔너리
    """
    print(f"\n[전체 평가] 데이터셋 평가 시작...")
    
    # 데이터 로드
    df = pd.read_csv(test_csv)
    base_dir = os.path.dirname(test_csv)
    
    # 결과 저장을 위한 자료구조
    results = defaultdict(list)
    predictions = []  # 전체 예측 리스트
    references = []   # 전체 정답 리스트
    
    # 타입별 통계를 위한 자료구조
    type_stats = defaultdict(lambda: {
        'total': 0,         # 전체 샘플 수
        'correct': 0,       # 정확히 맞춘 수
        'cer_sum': 0,       # CER 합계 (평균 계산용)
        'wer_sum': 0,       # WER 합계 (평균 계산용)
        'text_lengths': []  # 텍스트 길이 분포
    })
    
    print(f"총 {len(df)}개 샘플 평가 중...")
    
    # 프로그레스 바와 함께 각 샘플 평가
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="평가 진행"):
        # 이미지 경로와 정답 텍스트
        img_path = os.path.join(base_dir, row['image_path'])
        true_text = str(row['text'])  # 문자열로 변환 (안전성)
        
        # 예측 수행
        pred_text = self.predict(img_path)
        
        # 전체 예측/정답 리스트에 추가
        predictions.append(pred_text)
        references.append(true_text)
        
        # 정확도 및 오류율 계산
        is_correct = (pred_text == true_text)
        cer = self.calculate_cer(pred_text, true_text)
        wer = self.calculate_wer(pred_text, true_text)
        
        # 타입별 통계 업데이트
        text_type = row['type']  # 'printed' 또는 'handwritten'
        type_stats[text_type]['total'] += 1
        if is_correct:
            type_stats[text_type]['correct'] += 1
        type_stats[text_type]['cer_sum'] += cer
        type_stats[text_type]['wer_sum'] += wer
        type_stats[text_type]['text_lengths'].append(len(true_text))
        
        # 상세 결과 저장
        results['file'].append(row.get('original_file', ''))
        results['type'].append(text_type)
        results['true_text'].append(true_text)
        results['pred_text'].append(pred_text)
        results['is_correct'].append(is_correct)
        results['cer'].append(cer)
        results['wer'].append(wer)
        results['text_length'].append(len(true_text))
        
        # 필기체인 경우 세부 타입(글자/단어)도 저장
        if 'sub_type' in row and pd.notna(row['sub_type']):
            sub_type = row['sub_type']
            combined_type = f"{text_type}_{sub_type}"
            type_stats[combined_type]['total'] += 1
            if is_correct:
                type_stats[combined_type]['correct'] += 1
            type_stats[combined_type]['cer_sum'] += cer
            type_stats[combined_type]['wer_sum'] += wer
            type_stats[combined_type]['text_lengths'].append(len(true_text))
    
    # 전체 CER 계산
    if self.cer_metric:
        # Hugging Face 메트릭 사용
        try:
            overall_cer = self.cer_metric.compute(
                predictions=predictions,
                references=references
            )
        except:
            # 실패 시 수동 계산
            overall_cer = sum(results['cer']) / len(results['cer'])
    else:
        # 수동 계산
        overall_cer = sum(results['cer']) / len(results['cer'])
    
    # 전체 WER 계산
    overall_wer = sum(results['wer']) / len(results['wer'])
    
    # ===== 결과 출력 섹션 =====
    print("\n" + "="*60)
    print("[평가 결과 요약]")
    print("="*60)
    
    # 전체 성능 메트릭
    overall_accuracy = sum(results['is_correct']) / len(results['is_correct'])
    print(f"\n[전체 성능]")
    print(f"   - 정확도: {overall_accuracy:.2%} ({sum(results['is_correct'])}/{len(results['is_correct'])})")
    print(f"   - CER (Character Error Rate): {overall_cer:.4f}")
    print(f"   - WER (Word Error Rate): {overall_wer:.4f}")
    print(f"   - 평균 텍스트 길이: {np.mean(results['text_length']):.1f}자")
    
    # 결과를 DataFrame으로 변환
    results_df = pd.DataFrame(results)
    
    # CSV 파일로 저장 (선택적)
    if save_results:
        output_path = f"evaluation_results_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv"
        results_df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"\n[결과 저장] 상세 결과가 저장되었습니다: {output_path}")
    
    # ===== 오류 분석 섹션 =====
    self._analyze_errors(results_df)
    
    # 평가 결과 반환
    return {
        'overall_accuracy': overall_accuracy,
        'overall_cer': overall_cer,
        'overall_wer': overall_wer,
        'type_stats': dict(type_stats),
        'detailed_results': results_df
    }

def _analyze_errors(self, results_df):
    """
    오류 패턴 상세 분석 (내부 메서드)
    
    Args:
        results_df (pd.DataFrame): 평가 결과 DataFrame
    """
    print(f"\n[오류 분석]")
    
    # 오류만 필터링
    errors_df = results_df[~results_df['is_correct']]
    
    if len(errors_df) == 0:
        print("   모든 예측이 정확합니다!")
        return
    
    # 1. CER 기준 최악의 오류 케이스
    print("   * 최대 오류 케이스 (CER 기준 상위 5개):")
    worst_errors = errors_df.nlargest(min(5, len(errors_df)), 'cer')
    
    for idx, (_, row) in enumerate(worst_errors.iterrows(), 1):
        print(f"   {idx}. CER: {row['cer']:.4f}")
        # 긴 텍스트는 50자로 제한
        true_text = row['true_text'][:50] + '...' if len(row['true_text']) > 50 else row['true_text']
        pred_text = row['pred_text'][:50] + '...' if len(row['pred_text']) > 50 else row['pred_text']
        print(f"      실제: '{true_text}'")
        print(f"      예측: '{pred_text}'")
        print()
    
    # 2. 텍스트 길이별 오류 분포
    print("   * 텍스트 길이별 오류 분포:")
    length_bins = [0, 10, 20, 50, 100, float('inf')]
    bin_labels = ['0-10자', '10-20자', '20-50자', '50-100자', '100자 이상']
    
    for i in range(len(length_bins)-1):
        # 해당 길이 범위의 오류 필터링
        mask = (errors_df['text_length'] >= length_bins[i]) & (errors_df['text_length'] < length_bins[i+1])
        count = mask.sum()
        
        if count > 0:
            avg_cer = errors_df[mask]['cer'].mean()
            print(f"      {bin_labels[i]}: {count}개 오류 (평균 CER: {avg_cer:.4f})")
    
    # 3. 반복 패턴 감지
    print("\n   * 반복 패턴 분석:")
    repetition_count = 0
    
    for _, row in errors_df.iterrows():
        pred_text = row['pred_text']
        # 간단한 반복 감지: 2글자 이상이 3번 이상 반복
        for i in range(len(pred_text)-1):
            if i+6 <= len(pred_text):
                pattern = pred_text[i:i+2]
                if pred_text[i:i+6] == pattern * 3:
                    repetition_count += 1
                    break
    
    if repetition_count > 0:
        print(f"      반복 패턴이 감지된 오류: {repetition_count}개 ({repetition_count/len(errors_df)*100:.1f}%)")
        print("      → generate 함수의 no_repeat_ngram_size 파라미터 조정 필요")
    else:
        print("      반복 패턴이 감지되지 않았습니다.")

# 클래스에 메서드 추가
TrOCREvaluator.evaluate_full = evaluate_full
TrOCREvaluator._analyze_errors = _analyze_errors

print("전체 평가 및 오류 분석 메서드 추가 완료!")

전체 평가 및 오류 분석 메서드 추가 완료!


In [ ]:
# ============================================ 
# Cell 6: 시각화 메서드 추가
# ============================================

def visualize_results(self, results_df, title_prefix="TrOCR 모델"):
    """
    결과를 테이블 형식으로 출력 (matplotlib 대신 print 사용)
    
    Args:
        results_df (pd.DataFrame): 평가 결과 DataFrame
        title_prefix (str): 출력 제목 접두사
    """
    print("\n" + "="*80)
    print(f"{title_prefix} 평가 결과 (샘플 수: {len(results_df)})")
    print("="*80)
    
    # 1. 타입별 정확도 테이블
    print("\n[1. 타입별 정확도]")
    print("-" * 40)
    print(f"{'타입':^15} | {'정확도':^10} | {'샘플 수':^10}")
    print("-" * 40)
    
    type_acc = results_df.groupby('type')['is_correct'].agg(['mean', 'count'])
    for idx, (type_name, row) in enumerate(type_acc.iterrows()):
        accuracy = row['mean']
        count = row['count']
        print(f"{type_name:^15} | {accuracy:>9.2%} | {count:>10}")
    
    overall_acc = results_df['is_correct'].mean()
    print("-" * 40)
    print(f"{'전체':^15} | {overall_acc:>9.2%} | {len(results_df):>10}")
    
    # 2. CER 분포 통계
    print("\n[2. CER 분포 통계]")
    print("-" * 50)
    cer_values = results_df['cer']
    
    # 기본 통계량
    print(f"평균 CER: {cer_values.mean():.4f}")
    print(f"중앙값 CER: {cer_values.median():.4f}")
    print(f"표준편차: {cer_values.std():.4f}")
    print(f"최소값: {cer_values.min():.4f}")
    print(f"최대값: {cer_values.max():.4f}")
    
    # CER 구간별 분포
    print("\n[CER 구간별 분포]")
    print("-" * 35)
    print(f"{'CER 구간':^15} | {'샘플 수':^10} | {'비율':^8}")
    print("-" * 35)
    
    cer_bins = [0, 0.1, 0.3, 0.5, 1.0, float('inf')]
    cer_labels = ['0.0-0.1', '0.1-0.3', '0.3-0.5', '0.5-1.0', '1.0+']
    
    for i in range(len(cer_bins)-1):
        mask = (cer_values >= cer_bins[i]) & (cer_values < cer_bins[i+1])
        count = mask.sum()
        ratio = count / len(cer_values) * 100
        print(f"{cer_labels[i]:^15} | {count:>10} | {ratio:>7.1f}%")
    
    # 3. 텍스트 길이별 성능
    print("\n[3. 텍스트 길이별 성능]")
    print("-" * 50)
    print(f"{'길이 구간':^15} | {'평균 CER':^10} | {'샘플 수':^10}")
    print("-" * 50)
    
    length_bins = [0, 10, 20, 50, 100, float('inf')]
    length_labels = ['0-10자', '10-20자', '20-50자', '50-100자', '100자+']
    
    for i in range(len(length_bins)-1):
        mask = (results_df['text_length'] >= length_bins[i]) & (results_df['text_length'] < length_bins[i+1])
        if mask.sum() > 0:
            avg_cer = results_df[mask]['cer'].mean()
            count = mask.sum()
            print(f"{length_labels[i]:^15} | {avg_cer:>10.4f} | {count:>10}")
    
    # 4. 타입별 상세 통계
    print("\n[4. 타입별 상세 통계]")
    print("-" * 70)
    print(f"{'타입':^15} | {'평균 CER':^10} | {'평균 WER':^10} | {'정확도':^10} | {'샘플 수':^10}")
    print("-" * 70)
    
    type_stats = results_df.groupby('type').agg({
        'cer': 'mean',
        'wer': 'mean',
        'is_correct': 'mean',
        'file': 'count'
    })
    
    for type_name, row in type_stats.iterrows():
        print(f"{type_name:^15} | {row['cer']:>10.4f} | {row['wer']:>10.4f} | {row['is_correct']:>9.2%} | {row['file']:>10}")
    
    # 전체 통계
    print("-" * 70)
    overall_cer = results_df['cer'].mean()
    overall_wer = results_df['wer'].mean()
    overall_acc = results_df['is_correct'].mean()
    print(f"{'전체':^15} | {overall_cer:>10.4f} | {overall_wer:>10.4f} | {overall_acc:>9.2%} | {len(results_df):>10}")
    
    # 5. 성능 요약
    print("\n[5. 성능 요약]")
    print("-" * 50)
    print(f"완벽히 맞춘 샘플: {results_df['is_correct'].sum()}개 ({results_df['is_correct'].mean():.2%})")
    print(f"오류가 있는 샘플: {(~results_df['is_correct']).sum()}개 ({(~results_df['is_correct']).mean():.2%})")
    print(f"평균 텍스트 길이: {results_df['text_length'].mean():.1f}자")
    print("="*80)

# 클래스에 메서드 추가
TrOCREvaluator.visualize_results = visualize_results

print("시각화 메서드 추가 완료!")
print("TrOCREvaluator 클래스 정의 완료!")

시각화 메서드 추가 완료!
TrOCREvaluator 클래스 정의 완료!


In [40]:
# ============================================ 
# Cell 7: 평가기 초기화 및 샘플 테스트 실행
# ============================================

# 학습된 모델 경로 설정 (실제 경로로 변경 필요)
model_path = 'aihub-korean-trocr'  # 본인의 모델 경로로 변경하세요

# 평가기 초기화
print("=" * 60)
print("TrOCR 모델 평가 시작")
print("=" * 60)

evaluator = TrOCREvaluator(model_path)

# 빠른 샘플 테스트 (모델 작동 확인 + DataFrame 반환)
print("\n" + "="*60)
print("1. 샘플 테스트")
print("="*60)

sample_results = evaluator.test_samples(
    'trocr_dataset/validation.csv',    # 테스트 데이터 경로
    num_samples=20,                    # 시각화를 위해 샘플 수를 늘림
    return_df=True                     # DataFrame 반환 요청
)

print(f"\n샘플 테스트 완료!")
print(f"- 정확도: {sample_results['sample_accuracy']:.2%}")
if sample_results['detailed_results'] is not None:
    print(f"- 평균 CER: {sample_results['overall_cer']:.4f}")
    print(f"- 평균 WER: {sample_results['overall_wer']:.4f}")

TrOCR 모델 평가 시작
[모델 로딩] aihub-korean-trocr
[모델 로드 완료] 디바이스: cuda
[경고] CER 메트릭 로드 실패. 수동 계산 모드로 전환

1. 샘플 테스트

[샘플 테스트] 20개 샘플 테스트 시작

             파일명               |             실제 텍스트             |             예측 텍스트            
[O]  handwritten_1_글자_139_1393   |               객                |              객              
[O]  handwritten_2_단어_139_1394   |             초 저 녁              |            초저녁            
[O]  handwritten_2_단어_141_1414   |             그 렇 게              |            그렇게            
[O]  handwritten_2_단어_140_1404   |              육 체               |             육체             
[O]  handwritten_1_글자_139_1393   |               녀                |               녀              
[O]  handwritten_2_단어_142_1424   |            특 이 하 다             |           특이하다           
[O]  handwritten_2_단어_140_1404   |            나 타 나 다             |            나타나다           
[O]  handwritten_2_단어_140_1404   |            국 회 의 원             |          

In [32]:
# ============================================ 
# Cell 8: 샘플 데이터 시각화 (테이블 형식)
# ============================================

# 샘플 테스트 결과 시각화 (테이블 형식)
if sample_results['detailed_results'] is not None:
    print("\n[샘플 데이터 분석]")
    evaluator.visualize_results(
        sample_results['detailed_results'],
        title_prefix="TrOCR 모델 (샘플 20개)"
    )
else:
    print("시각화할 데이터가 없습니다.")


[샘플 데이터 분석]

TrOCR 모델 (샘플 20개) 평가 결과 (샘플 수: 20)

[1. 타입별 정확도]
----------------------------------------
      타입        |    정확도     |    샘플 수   
----------------------------------------
  handwritten   |    95.00% |       20.0
----------------------------------------
      전체        |    95.00% |         20

[2. CER 분포 통계]
--------------------------------------------------
평균 CER: 0.0500
중앙값 CER: 0.0000
표준편차: 0.2236
최소값: 0.0000
최대값: 1.0000

[CER 구간별 분포]
-----------------------------------
    CER 구간      |    샘플 수    |    비율   
-----------------------------------
    0.0-0.1     |         19 |    95.0%
    0.1-0.3     |          0 |     0.0%
    0.3-0.5     |          0 |     0.0%
    0.5-1.0     |          0 |     0.0%
     1.0+       |          1 |     5.0%

[3. 텍스트 길이별 성능]
--------------------------------------------------
     길이 구간      |   평균 CER   |    샘플 수   
--------------------------------------------------
     0-10자      |     0.0500 |         20

[4. 타입별 상세 통계]
----------

In [33]:
# ============================================ 
# Cell 9: 전체 데이터셋 평가 (선택사항)
# ============================================

# 전체 데이터셋 평가 - 시간이 오래 걸리므로 선택적으로 실행
# 주석을 해제하여 실행하세요

print("\n" + "="*60)
print("2. 전체 데이터셋 평가")
print("="*60)
print("전체 평가는 시간이 오래 걸립니다.")
print("실행하려면 아래 주석을 해제하세요:")
print()

# full_results = evaluator.evaluate_full('trocr_dataset/validation.csv')
# 
# # 전체 결과 시각화
# if full_results['detailed_results'] is not None:
#     evaluator.visualize_results(
#         full_results['detailed_results'],
#         title_prefix="TrOCR 모델 (전체 데이터)"
#     )


2. 전체 데이터셋 평가
전체 평가는 시간이 오래 걸립니다.
실행하려면 아래 주석을 해제하세요:

